In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

#change path
root_dir = "/content/drive/My Drive/"
base_dir = root_dir + 'python_lesson/CloneCoding'
os.chdir(base_dir)

#mount google driver
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
#root_dir = "/content/gdrive/My Drive/"

!pwd

/content/drive/My Drive/python_lesson/CloneCoding


# 영화 관객 예측
***


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings('ignore')

In [4]:
train = pd.read_csv("./영화 관객수 예측 모델 개발/data/movies_train.csv")
test = pd.read_csv('./영화 관객수 예측 모델 개발/data/movies_test.csv')
submission = pd.read_csv('./영화 관객수 예측 모델 개발/data/submission.csv')

In [5]:
train.head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083


In [6]:
test.head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,3.005290e+05,2,304,3
1,아빠가 여자를 좋아해,(주)쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,3.427002e+05,4,275,3
2,하모니,CJ 엔터테인먼트,드라마,2010-01-28,115,12세 관람가,강대규,4.206611e+06,3,419,7


In [7]:
submission.head(3)

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0


#### 배급사 전처리

In [8]:
# distributor 컬럼에서 '주'가 붙은거 공백으로 변경해주기
train['distributor'] = train.distributor.str.replace("(주)", '')
test['distributor'] = test.distributor.str.replace("(주)", '')

In [9]:
# distributor 컬럼에서 '주'가 빠진 후 남은 '()' 공백으로 변경해주기
train['distributor'] = [re.sub(r'[^0-9a-zA-Z가-힣]', '', x) for x in train.distributor]
test['distributor'] = [re.sub(r'[^0-9a-zA-Z가-힣]', '', x) for x in test.distributor]

In [10]:
def get_dis(x) :
    if 'CJ' in x or 'CGV' in x :
        return 'CJ'
    elif '쇼박스' in x :
        return '쇼박스'
    elif 'SK' in x :
        return 'SK'
    elif '리틀빅픽' in x :
        return '리틀빅픽처스'
    elif '스폰지' in x :
        return '스폰지'
    elif '싸이더스' in x :
        return '싸이더스'
    elif '에이원' in x :
        return '에이원'
    elif '마인스' in x :
        return '마인스'
    elif '마운틴픽' in x :
        return '마운틴픽처스'
    elif '디씨드' in x :
        return '디씨드'
    elif '드림팩트' in x :
        return '드림팩트'
    elif '메가박스' in x :
        return '메가박스'
    elif '마운틴' in x :
        return '마운틴'
    else :
        return x

In [11]:
train['distributor'] = train.distributor.apply(get_dis)
test['distributor'] = test.distributor.apply(get_dis)

#### 장르별 영화 관객수 평균값으로 랭크 인코딩

In [12]:
train.groupby('genre').box_off_num.mean().sort_values()
# train[['genre', 'box_off_num']].groupby('genre').mean().sort_values('box_off_num') (다른 표현)

genre
뮤지컬       6.627000e+03
다큐멘터리     6.717226e+04
서스펜스      8.261100e+04
애니메이션     1.819267e+05
멜로/로맨스    4.259680e+05
미스터리      5.275482e+05
공포        5.908325e+05
드라마       6.256898e+05
코미디       1.193914e+06
SF        1.788346e+06
액션        2.203974e+06
느와르       2.263695e+06
Name: box_off_num, dtype: float64

In [13]:
# 위에서 구한 장르 순위 정보를 컬럼으로 추가
train['genre_rank'] = train.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})
test['genre_rank'] = test.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})

#### 배급사별 영화 관객수 중위값 기준으로 배급사 랭크 인코딩

In [14]:
# 여기서 평균을 사용하지 않고, 중위값을 사용한 이유는 배급사별 영화 관객수 편차가 커서 그런 것으로 생각됨.
tr_nm_rank = train.groupby('distributor').box_off_num.median().reset_index(name = 'num_rank').sort_values(by = 'num_rank')
tr_nm_rank

,distributor,num_rank
110,인피니티엔터테인먼트,2.0
15,고구마공작소,8.0
52,사람과사람들,42.0
97,위드시네마,46.0
19,나우콘텐츠,54.0
...,...,...
113,전망좋은영화사,1214237.0
105,이십세기폭스코리아,1422844.0
56,쇼박스,2138560.0
84,영구아트무비,2541603.0


In [15]:
# 오름차순으로 정렬되어 있기 때문에 배급사별 랭크를 매겨줌.
tr_nm_rank['num_rank'] = [i + 1 for i in range(tr_nm_rank.shape[0])]

In [16]:
tr_nm_rank

,distributor,num_rank
110,인피니티엔터테인먼트,1
15,고구마공작소,2
52,사람과사람들,3
97,위드시네마,4
19,나우콘텐츠,5
...,...,...
113,전망좋은영화사,143
105,이십세기폭스코리아,144
56,쇼박스,145
84,영구아트무비,146


#### 최종 데이터 병합

In [17]:
# train 데이터에 배급사 별 관객 랭크 순위 데이터 추가
train = pd.merge(train, tr_nm_rank, how = 'left')

In [18]:
test = pd.merge(test, tr_nm_rank, how = 'left')

In [19]:
test.fillna(0, inplace = True)

In [20]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.2 MB/s 


In [21]:
!pip install ngboost

     |████████████████████████████████| 348 kB 8.1 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4049 sha256=c90dc960a5c09822b2c9c212ccc54cd3dd14d25b9dcce40ab759ea72b4d42958
  Stored in directory: /root/.cache/pip/wheels/9f/01/ee/1331593abb5725ff7d8c1333aee93a50a1c29d6ddda9665c9f
Successfully built autograd-gamma


In [22]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from ngboost import NGBRegressor

#### 모델링 데이터 구성
- 타겟 값인 관객수를 로그변환
- 상영등급은 더미 변수
- 출연 배우 수는 로그변환

---
✨ log 변환은 정규분포 형성하도록 변환해주는 것임.

In [23]:
X = train[['num_rank', 'time', 'num_staff', 'num_actor', 'genre_rank', 'screening_rat']]
y = np.log1p(train.box_off_num)

In [24]:
X = pd.get_dummies(columns = ['screening_rat'], data = X) # 상영등급은 원-핫 인코딩 형식으로 변경

In [25]:
X['num_actor'] = np.log1p(X['num_actor'])

In [26]:
target = test[['num_rank', 'time', 'num_staff', 'num_actor', 'genre_rank', 'screening_rat']]

In [27]:
target = pd.get_dummies(columns = ['screening_rat'], data = target)

In [28]:
target['num_actor'] = np.log1p(target['num_actor'])

In [29]:
target

,num_rank,time,num_staff,num_actor,genre_rank,screening_rat_12세 관람가,screening_rat_15세 관람가,screening_rat_전체 관람가,screening_rat_청소년 관람불가
0,132.0,125,304,1.386294,12,0,0,0,1
1,145.0,113,275,1.386294,5,1,0,0,0
2,141.0,115,419,2.079442,8,1,0,0,0
3,145.0,116,408,1.098612,11,0,1,0,0
4,141.0,110,380,0.693147,7,0,1,0,0
...,...,...,...,...,...,...,...,...,...
238,112.0,78,4,1.609438,8,0,1,0,0
239,69.0,86,18,1.098612,2,1,0,0,0
240,0.0,107,224,1.609438,12,0,0,0,1
241,46.0,102,68,2.079442,8,0,0,0,1


In [30]:
test

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_rank,num_rank
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,3.005290e+05,2,304,3,12,132.0
1,아빠가 여자를 좋아해,쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,3.427002e+05,4,275,3,5,145.0
2,하모니,CJ,드라마,2010-01-28,115,12세 관람가,강대규,4.206611e+06,3,419,7,8,141.0
3,의형제,쇼박스,액션,2010-02-04,116,15세 관람가,장훈,6.913420e+05,2,408,2,11,145.0
4,평행 이론,CJ,공포,2010-02-18,110,15세 관람가,권호영,3.173800e+04,1,380,1,7,141.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,해에게서 소년에게,디씨드,드라마,2015-11-19,78,15세 관람가,안슬기,2.590000e+03,1,4,4,8,112.0
239,울보 권투부,인디스토리,다큐멘터리,2015-10-29,86,12세 관람가,이일하,0.000000e+00,0,18,2,2,69.0
240,어떤살인,컨텐츠온미디어,느와르,2015-10-28,107,청소년 관람불가,안용훈,0.000000e+00,0,224,4,12,0.0
241,말하지 못한 비밀,마운틴픽처스,드라마,2015-10-22,102,청소년 관람불가,송동윤,5.069900e+04,1,68,7,8,46.0


In [31]:
X

,num_rank,time,num_staff,num_actor,genre_rank,screening_rat_12세 관람가,screening_rat_15세 관람가,screening_rat_전체 관람가,screening_rat_청소년 관람불가
0,134,96,91,1.098612,11,0,0,0,1
1,145,130,387,1.386294,12,0,0,0,1
2,145,123,343,1.609438,11,0,1,0,0
3,140,101,20,1.945910,9,0,0,1,0
4,145,108,251,1.098612,9,0,1,0,0
...,...,...,...,...,...,...,...,...,...
595,140,111,510,2.079442,8,0,0,0,1
596,145,127,286,1.945910,8,0,1,0,0
597,46,99,123,1.609438,7,0,0,0,1
598,141,102,431,1.609438,12,0,1,0,0


In [32]:
y

0      10.060449
1      15.771725
2      15.755558
3      12.291640
4      13.088575
         ...    
595    14.204231
596    14.355762
597     7.814400
598    14.600565
599    15.784933
Name: box_off_num, Length: 600, dtype: float64

#### 10Fold로 교차검증

In [33]:
kf = KFold(n_splits =  10, shuffle = True, random_state = 42)

### 5가지 모델 사용
#### 1) <a href=https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html>GradientBoostingRegressor</a>

**GradientBoostingRegressor**은 앙상블 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가하지만 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습시킨다.

다양한 매개변수가 존재한다.

- loss: 최적화할 손실 함수를 선택하는 것이다. 'ls'는 최소 제곱(Least Square)회귀를 나타내고, 'lad'(최소 절대 편차)는 입력 변수의 순서 정보만을 기반으로 하는 손실 함수이다. 'huber'는 이 둘의 조합이며, 'quantile'은 분위수 회귀를 허용한다. (default='ls')
- **learning_rate**: 각 트리의 기여도를 나타낸다. (default=0.1)
- **n_estimators**: 부스팅 단계의 수를 나타낸다. (default=100)
- subsample: 샘플 비율을 나타냄. 1보다 작으면 확률적 그레디언트 부스팅이 발생.(default=1.0)
- criterion: 분할하는데의 기준을 선택하는 것이다. 'friedman_mse', 'mse', 'mae'가 존재. (default='friedman_mse')

In [34]:
gbm = GradientBoostingRegressor(random_state = 42)

In [44]:
rmse_list = []
gb_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    gbm.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in gbm.predict(val_x)]) # np.expm1: exp()-1
    sub_pred = np.expm1([0 if x < 0 else x for x in gbm.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    gb_pred += (sub_pred / 10)

In [45]:
tr_idx

array([  0,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  14,
        15,  16,  17,  18,  19,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        72,  73,  74,  75,  76,  77,  78,  79,  81,  82,  83,  84,  85,
        86,  88,  89,  90,  92,  93,  94,  95,  96,  97,  98, 100, 101,
       103, 104, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 131, 132,
       133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,
       146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,
       159, 161, 162, 163, 164, 165, 167, 168, 169, 170, 171, 172, 173,
       174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186,
       187, 188, 190, 192, 193, 194, 195, 196, 197, 198, 199, 20

In [46]:
rmse_list

[833799.7898958934,
 843326.1703815515,
 2580094.3543833084,
 1015979.3764714784,
 1548450.3688728388,
 1041278.3298725794,
 811791.6099649561,
 798780.6574496045,
 1294844.9626026177,
 975444.1672700263]

In [47]:
np.mean(rmse_list)

1174378.9787164854

#### 2) <a href=https://stanfordmlgroup.github.io/projects/ngboost/>NGBRegressor</a>

**Natural Gradient Boost**(NGBoost)은 XGBoost나 LightBoost보다 약간 더 나은 성능을 보여주는 기법입니다. <br>
- 특징으로 확률적인 예측을 해줍니다.(예측의 불확실성 측정해줌.)
  - 일반적인 모델을 활용하면, '집의 위치, 주변 환경, 평수, 건축 연도 등을 고려해 5억이다' 라고 예측을 할 것이다. 예측은 언제나 불확실성을 내포합니다. 사용한 모델이 5억이라고 예측했을 때 신뢰성이 얼마나 되는지 알 수 없다. 하지만 NGBoost는 예측의 불확실성까지 알려주기 때문에 '집의 위치, 주변 환경, 평수, 건축 년도 등을 고려해 5억일 확률은 75%다' 라고 알려줌.
- **즉, 응답에 대한 확률 분포를 알려줌.**
- 기본적인 학습기는 결정 트리(Decision Tree)를 사용하며, Input을 받아 약 학습기(Weak Learner)로 예측한 결과를 활용해 확률 분포를 생성한다.
- 예측한 확률 분포와 실제 타겟 값을 토대로 성능 측정을 하고, 성능 측정을 위한 알고리즘은 MLE이나 CRPS를 주로 사용한다.
- 위 단계를 반복해 성능을 높여가는 방식이다.

In [48]:
ngb = NGBRegressor(random_state = 518)

In [49]:
rmse_list = []
ngb_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    ngb.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in ngb.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in ngb.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    ngb_pred += (sub_pred / 10)

[iter 0] loss=2.6200 val_loss=0.0000 scale=1.0000 norm=2.9523
[iter 100] loss=1.9566 val_loss=0.0000 scale=2.0000 norm=2.7106
[iter 200] loss=1.5812 val_loss=0.0000 scale=2.0000 norm=2.1869
[iter 300] loss=1.4423 val_loss=0.0000 scale=2.0000 norm=2.0889
[iter 400] loss=1.3728 val_loss=0.0000 scale=1.0000 norm=1.0177
[iter 0] loss=1.4091 val_loss=0.0000 scale=1.0000 norm=1.0766
[iter 100] loss=1.3282 val_loss=0.0000 scale=1.0000 norm=1.0099
[iter 200] loss=1.2863 val_loss=0.0000 scale=0.5000 norm=0.4934
[iter 300] loss=1.2458 val_loss=0.0000 scale=1.0000 norm=0.9650
[iter 400] loss=1.2117 val_loss=0.0000 scale=0.5000 norm=0.4743
[iter 0] loss=1.2136 val_loss=0.0000 scale=1.0000 norm=0.9547
[iter 100] loss=1.1615 val_loss=0.0000 scale=0.2500 norm=0.2295
[iter 200] loss=1.1509 val_loss=0.0000 scale=0.1250 norm=0.1139
[iter 300] loss=1.1436 val_loss=0.0000 scale=0.0625 norm=0.0566
[iter 400] loss=1.1403 val_loss=0.0000 scale=0.0005 norm=0.0004
[iter 0] loss=1.1710 val_loss=0.0000 scale=1.0

In [50]:
np.mean(rmse_list)

1406727.6583156916

#### 3) <a href=https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.Booster.html>LGBMRegressor</a>

**Light Gradient Boosting Machine**(light GBM)은 Gradient Boosting 프레임워크로 Tree 기반 학습 알고리즘 입니다.<br>
기존의 Tree 기반 알고리즘과 다르게, Light GBM은 Tree가 수직적으로 확장됩니다.(leaf-wise) 확장하기 위해 max delta loss를 가진 leaf를 선택함으로써, 동일한 leaf를 확장할 때, leaf-wise 알고리즘은 level-wise 알고리즘보다 더 많은 loss, 손실을 줄일 수 있다. <br>
- Light GBM은 큰 사이즈의 데이터를 다룰 수 있으며 실행시킬 때 적은 메모리를 차지합니다. 게다가 속도도 매우 빠릅니다.
- 하지만 작은 데이터 세트에 사용될 경우, 오버피팅에 민감해집니다.(보통 10,000 이상의 row 데이터에서 사용을 권장)
- Light GBM은 매우 많은 파라미터를 가지고 있음. 대표적인 파라미터 소개
  - max_depth: Tree 최대 깊이.(모델 과적합 다룰 때 사용)
  - min_data_in_leaf: Leaf가 가지고 있는 최소한의 레코드 수. (Default = 20). 과적합 해결할 때 사용되는 파라미터임.
  - feature_fraction: Boosting이 랜덤 포레스트일 경우 사용.(0.8 feature_fraction의 의미는 Light GBM이 Tree를 만들 때 매번 각각의 iteration 마다 파라미터 중에서 80%를 랜덤하게 선택하는 것을 의미.)
  - bagging_fraction: 매번 iteration을 둘 때 사용되는 데이터의 일부를 선택하는데, 훈련 속도를 높여주며 과적합을 방지할 때 주로 사용됨.
  - early_stopping_round: 분석 속도를 높이는데 도움이 됨. 만약 어떤 validation 데이터 중 하나의 지표가 지난 early_stopping_round 에서 향상되지 않았다면 학습을 중단. 지나친 iteration을 줄이는데 도움이 됨.
  - lambda: regularization을 의미. 일반적인 값의 범위는 0에서 1사이임.
  - num_leaves: 전체 Tree의 leave 수. (Default=31)
  - learning_rate: 최종 결과에 대한 각각의 Tree에 영향을 미침. (보통 0.1, 0.001, 0.003 etc..)
  - num_boost_round: boosting iteration 수 (일반적으로 100 이상)
  - boosting:실행하고자 하는 알고리즘 타입 정의(Default: gdbt)
    - gdbt: Traditional Gradient Boosting Decision Tree
    - rf: Random Forest
    - dart: Dropouts meet Multiple Additive Regression Trees
    - goss: Gradient-based One-Side Sampling
  - application: 가장 중요한 파라미터, regression(회귀 분석 문제)인지 classification(분류 분석 문제)인지를 정함. (Default: regression)
    - regression: 회귀 문제
    - binary: 이진 분류
    - multiclass: 다중 분류
  - metric: 모델 구현 시 손실을 정함.
    - mae: mean absolute error
    - mse: mean squared error
    - binary_logloss: loss for binary classificaion
    - multi_logloss: loss for multi classification
    

In [51]:
lgbm = LGBMRegressor(random_state = 518)

In [52]:
rmse_list = []
lgb_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    lgbm.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in lgbm.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in lgbm.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    lgb_pred += (sub_pred / 10)

In [53]:
np.mean(rmse_list)

1213815.0440528719

#### 4) <a href=https://xgboost.readthedocs.io/en/stable/>XGBRegressor</a>

**Extreme Gradient Boosting**(XGBoost)은 병렬 학습이 지원되도록 구현한 기법이다.
- 병렬 처리로 학습하기 때문에, 분류 속도가 빠르다.
- 표준 GBM 경우 과적합 규제 기능이 없으나, XGBoost는 자체에 과적합 규제 기능을 가짐.
- 분류와 회귀 문제 모두 지원하며, 성능과 저원 효율이 좋아 빈번하게 사용되는 알고리즘임.
- 파라미터
  - booster: 어떤 boost 구조를 쓸지 결정.(Default: gbtree)
    - gbtree: 의사결정 기반 모형
    - gblinear: 선형 모형
    - dart
  - n_jobs: 실행하는데 사용되는 병렬 스레드 수
  - n_estimators: 생성할 weak learner의 수(Default: 100). learning_rate가 낮을 땐, n_estimators를 높여야 과적합이 방지됨.
  - max_depth: 트리의 maximum depth.(Default: 6) 보통 3에서 10 사이 값 적용.
  - subsample: weak learner가 학습에 사용하는 데이터 샘플링 비율. (Default: 1) 보통 0.5에서 1사이 사용. 값이 낮을수록 과적합이 방지.
  

In [54]:
xgb = XGBRegressor(random_state = 518)

In [55]:
rmse_list = []
xgb_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    xgb.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in xgb.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in xgb.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    xgb_pred += (sub_pred / 10)

[13:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [56]:
np.mean(rmse_list)

1002219.3081045212

#### 5) <a href=https://catboost.ai/>CatBoostRegressor</a>

**CatBoost**의 특징
- Level-wise로 트리 생성.
- Orderd Boosting: 기존 부스팅 모델은 일괄적으로 모든 훈련 데이터를 대상으로 잔차계산 수행하였으나, CatBoost는 일부만 가지고 잔차계산을 한 뒤, 모델 생성, 그 뒤에 데이터의 잔차는 이 모델로 예측한 값을 사용.(이해가 안되면, 아래 과정을 보면 됨.)
  - 기존 부스팅 기법은 모든 데이터 ($x_1 \sim x_{10}$)까지의 잔차를 일괄 계산함. 반면, Catboost의 과정은 다음과 같음.
  - 1. 먼저 $x_1$의 잔차만 계산, 이를 기반으로 모델 생성. 그리고 $x_2$의 잔차를 해당 모델로 예측.
  - 2. $x_1, x_2$의 잔차를 가지고 모델 생성. 이를 기반으로 $x_3, x_4$의 잔차를 모델로 예측.
  - 3. $x_1,x_2,x_3,x_4$를 가지고 모델 생성. 이를 기반으로 $x_5, x_6, x_7, x_8$의 잔차를 모델로 예측.
  - 4. ... 반복
- Random Permutaion: Ordered Boosting을 할 때, 데이터 순서를 섞어주지 않으면 매번 같은 순서대로 잔차를 예측하는 모델을 만들 가능성이 있음. 따라서 데이터를 셔플링해서 뽑아냄. 과적합을 방지하기 위해, 트리를 다각적으로 만들려는 의미.
- Ordered Target Encoding: 범주형 변수를 numeric으로 인코딩.
  - <img width=400 src=https://user-images.githubusercontent.com/48666867/148556093-634c9ee2-560e-4df5-9c66-d605df445979.png>
  - 예를 들어, class_label의 값의 평균으로 인코딩.(cloudy 클래스를 가진 기온 데이터 (15+25+20+20)/4=20의 값으로 인코딩.) 이를 Mean encoding이라 함.
  - 하지만, 이는 예측해야 한느 값이 훈련 셋 피처에 들어가버리는 문제, 즉 Data Leakage 문제를 일으킴. (과적합의 주요 원인)
  -  CatBoost는 이를 해결하기 위해, 현재 데이터의 인코딩을 하기 전에 이전 데이터들의 인코딩된 값을 사용함.
  - Ex) Friday에는, cloudy=(15+14)/2 = 15.5로 인코딩.
  - Ex) Saturday에는, cloudy=(15+14+20)/3 = 16.3로 인코딩.
- Categorical Feature Combinations: information gain이 동일한 features를 하나의 feature로 묶어버림. 결과적으로, 데이터 전처리에 있어 feature selection 부담이 조금 줄어듬.
  - 데이터의 대부분이 수치형 변수인 경우, Light GBM보다 학습 속도가 느림.(데이터 대부분이 범주형 변수인 경우 쓰면 유용함)


In [57]:
cat = CatBoostRegressor(random_state = 518, silent = True)

In [58]:
rmse_list = []
cat_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    cat.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in cat.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in cat.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    cat_pred += (sub_pred / 10)

In [59]:
np.mean(rmse_list)

1075600.9209088846

#### 6) <a href=https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html>RandomForestRegressor</a>

**RandomForest**는 dataset의 다양한 하위 sample에 대한 여러 개의 Decision Tree에서의 평균을 사용하여 predict Accuracy를 높이고 과적합을 제어하는 meta estimator이다.
- n_estimators: 랜덤 포레스트 트리의 수(Default: 100)
- criterion: 트리 구분 기준. mse or mae.(Default: 'mse')
- max_depth: 트리 최대 깊이.(Default=None) None일 경우, 모든 잎이 순수해질 때까지나 min_samples_split 샘플 미만이 포함될때까지 노드가 확장됨.
- min_sample_split: 내부 노드를 분할하는데 필요한 최소 샘플 수.(Default:2)
- min_sample_leaf: 리프 노드에 있어야하는 최소 샘플 수. 왼쪽과 오른쪽 분기 각각에 최소한 훈련 샘플을 남겨두는 경우에 고려되며, 회귀에서 모델을 평활화하는데 효과가 있을 수 있음.

In [60]:
rf = RandomForestRegressor(random_state = 518)

In [61]:
rmse_list = []
rf_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    rf.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in rf.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in rf.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    rf_pred += (sub_pred / 10)

In [62]:
np.mean(rmse_list)

873671.7029559091

***

#### 최종 예측 결과 블렌딩

In [63]:
submission['box_off_num'] = (xgb_pred + cat_pred + lgb_pred + rf_pred + gb_pred + ngb_pred) / 6

In [64]:
submission.sort_values(by = 'box_off_num')

,title,box_off_num
130,댄서김의 은밀한 교수법,1.246101e+01
39,REC 알이씨,1.493858e+01
139,화려한 외출,2.031685e+01
173,옹녀뎐,2.196576e+01
65,엄마는 창녀다,2.783574e+01
...,...,...
135,용의자,4.131189e+06
229,베테랑,4.304552e+06
142,박수건달,5.024071e+06
179,군도: 민란의 시대,6.384855e+06


# Reference
1. https://dacon.io/competitions/open/235536/codeshare/2721?page=1&dtype=recent
2. https://biology-statistics-programming.tistory.com/49
3. https://hanishrohit.medium.com/whats-so-special-about-catboost-335d64d754ae
4. https://nurilee.com/2020/04/03/lightgbm-definition-parameter-tuning/
5. https://wooono.tistory.com/97
6. https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-16-NGBoost